# Handling board and detecting winners

## Idea 1: Bitwise Operations

- `Game` or `Board` object stores current state as N x N matrix, which allows easy setting/getting (board[x,y])
- After each move, a method converts this to a binary string, where 1's represent positions held by current player.

```python
    board = [['X', 'O', ' '],
             ['O', 'X', 'O'],
             [' ', 'X', ' ']] # -> To be encoded as `100010010`
```     

- The encoded string is compared to each of the 8 winning string operations using bitwise operation: `cur_position & 111000000 == 111000000`
- If there's a match, we have a winner

In [1]:
winners = [
    0b111000000,
    0b000111000,
    0b000000111,
    0b100100100,
    0b010010010,
    0b001001001,
    0b100010001,
    0b001010100
]

In [2]:
cur_position = 0b010000011

def check(cur_position):
    for winner in winners:
        if winner & cur_position == winner:
            print('Winner')
            return
    print('no winner')
    return

check(cur_position)

no winner


In [3]:
board = [['X', 'O', '_'],
         ['X', '_', 'O'],
         ['O', 'X', 'X']]

board

[['X', 'O', '_'], ['X', '_', 'O'], ['O', 'X', 'X']]

In [4]:
def flatten(board):
    return [item for row in board for item in row]

def encode(flat_board, playerchar):
    binseq = '0b'
    for char in flat_board:
        if char == playerchar:
            binseq += '1'
        else:
            binseq += '0'
    return int(binseq, 2)

In [5]:
board = [['X', 'O', '_',],
         ['X', 'X', 'O'],
         ['O', 'O', 'X']]

check(encode(flatten(board), 'X'))

Winner


In [6]:
def newboard(): return [['', '', ''],['', '', ''], ['','','']]

board = newboard()

board[0][1] = 'X'
board[0][0] = 'X'
board[0][2] = 'X'

check(encode(flatten(board), 'X'))

Winner


# Idea 2: Summing Rows, Columns, and Diagonals (from scratch)
- Board initialized with zeroes.
- Board stores `1` for player 1 moves and `-1` for player 2 moves. A row, column, or diagonal that sums to 3 is a win for player 1, -3 a win for player 2.
- To simpllify checking, check only rows and diagonal and then transpose matrix and recheck
- One advantage is that it should make it easier to handle NxN games of tic-tac-toe

In [7]:
board = [[1, -1, 0], [0, 0, 1], [-1, -1, 1]]
board

[[1, -1, 0], [0, 0, 1], [-1, -1, 1]]

In [8]:
def sumrows(board):
    rowresults = []
    for row in board:
        result = 0
        for col in row:
            result += col
        rowresults.append(result)
    return rowresults

def sumdiag(board):
    n = len(board)
    result = 0
    for i in range(n):
        result += board[i][i]
    return result

print('Rows: {}'.format(sumrows(board)))
print('Diagonal: {}'.format(sumdiag(board)))

Rows: [0, 1, -1]
Diagonal: 2


In [9]:
def transpose(board):
    newboard = [['','',''],['','',''],['','','']]
    for row in range(len(board)):
        for col in range(len(board[row])):
            newboard[row][col] = board[col][row]
    return newboard

def reverse_row_inplace(row):
    l = 0
    r = len(a) - 1
    while l < r:
        row[l], row[r] = row[r], row[l]
        l += 1
        r -= 1
    return row

def reverse_row(row):
    newrow = []
    i = len(row) - 1
    while i >= 0:
        newrow.append(row[i])
        i -= 1
    return newrow

def mirror(board):
    newboard = []
    for row in board:
        newboard.append(reverse_row(row))
    return newboard

def rotate90(board):
    return transpose(mirror(board))

In [10]:
print('Columns: {}'.format(sumrows(rotate90(board))))
print('Reverse Diagonal: {}'.format(sumdiag(rotate90(board))))

Columns: [2, -2, 0]
Reverse Diagonal: -1


# Idea 3: Same as Above, but with NumPy

In [11]:
import numpy as np

In [12]:
board = np.array([[1, -1, 0], [0, 0, 1], [-1, -1, 1]])
board

array([[ 1, -1,  0],
       [ 0,  0,  1],
       [-1, -1,  1]])

In [13]:
# Sum diagonal
board.diagonal().sum()

2

In [14]:
# Sum rows
board.sum(axis=1)

array([ 0,  1, -1])

In [15]:
# Sum cols
board.sum(axis=0)

array([ 0, -2,  2])

In [16]:
# Sum anti-diagonal
np.fliplr(board).diagonal().sum()

-1